In [14]:
# In this notebook we will be taking a look at the breast cancer dataset, and will be writing an Support Vector Machine Algo from "scratch".
import numpy as np 
import pandas as pd  
import statsmodels.api as sm
from sklearn.preprocessing import MinMaxScaler 
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score
from sklearn.utils import shuffle

In [21]:
# The Cost Function: An SVM seeks to minimize this function. We are seeking a hyperplane (n-dimension surface)
# that seperates the postive cases from the negative cases while minimizing misclassification. The cost function is 
# more or less proportional to the quality of the model. The ruling equation is J(w) ~ w^2 + (C/N)*nSUMi[1-yi(w*xi+b)] 

def compute_cost(W, X, Y):
    # Here we will attempt to compute the cost by starting with the hinge loss
    N = X.shape[0]
    distances = 1 - Y * (np.dot(X,W))
    distances[distances < 0] = 0
    hinge_loss = reg_strength * (np.sum(distances) / N)

    cost = 1/2 * np.dot(W, W) + hinge_loss

    return cost

# The Gradient of J(w) ~(1/N)*nSUMi[w when 1-yi(w*xi+b), else w-C{yi,xi}]
def calculate_cost_gradient(W, X_batch, Y_batch):
    if type(Y_batch) == np.float64:
        Y_batch = np.array([Y_batch])
        X_batch = np.array([X_batch])
    
    distance = 1 - (Y_batch * np.dot(X_batch, W))
    dw = np.zeros(len(W))
    
    for ind, d in enumerate(distance):
        if max(0, d) == 0:
            di = W
        else:
            di = W - (reg_strength * Y_batch[ind] * X_batch[ind])
        
        dw += di
    
    dw = dw/len(Y_batch)  # average
    
    return dw

# Training using Stochastic Gradient Descent (SGD), 
# we will seek to minimize J(w) which means minimize w^2 and minimize hinge-loss. 
# We will do so by moving opposite of the gradient (this is the Descent part) (w') until convergence w' = min(J(w))
def sgd(features, outputs):
    max_epochs = 5000
    weights = np.zeros(features.shape[1])
    nth = 0
    prev_cost = float("inf")
    cost_threshold = 0.01 #percentage
    
    for epoch in range(1, max_epochs):
        X, Y = shuffle(features, outputs)
        
        for ind, x in enumerate(X):
            ascent = calculate_cost_gradient(weights, x, Y[ind])
            weights = weights - (learning_rate * ascent)
        
        # convergence check on 2^nth epoch
        if epoch == 2 ** nth or epoch == max_epochs - 1:
            cost = compute_cost(weights, features, outputs)
            print("Epoch is:{} and Cost is: {}".format(epoch, cost))
            # stoppage criterion
            if abs(prev_cost - cost) < cost_threshold * prev_cost:
                return weights
            prev_cost = cost
            nth += 1    
    return weights

# V2:
def remove_correlated_features(X):
    corr_threshold = 0.9
    corr = X.corr()
    drop_columns = np.full(corr.shape[0], False, dtype=bool)
    
    for i in range(corr.shape[0]):
        for j in range(i + 1, corr.shape[0]):
            if corr.iloc[i, j] >= corr_threshold:
                drop_columns[j] = True
    
    columns_dropped = X.columns[drop_columns]
    X.drop(columns_dropped, axis=1, inplace=True)
    
    return columns_dropped

# V2:
def remove_less_significant_features(X, Y):
    # define a Significance Level "sl"
    sl = 0.05
    regression_ols = None
    columns_dropped = np.array([])
    
    for itr in range(0, len(X.columns)):
        regression_ols = sm.OLS(Y, X).fit()
        max_col = regression_ols.pvalues.idxmax()
        max_val = regression_ols.pvalues.max()
        if max_val > sl:
            X.drop(max_col, axis='columns', inplace=True)
            columns_dropped = np.append(columns_dropped, [max_col])
        else:
            break
    
    regression_ols.summary()
    
    return columns_dropped

def init():
    # Explore the dataset
    data = pd.read_csv('breast_cancer_data.csv')
    data.describe()
    
    # Pre-Processing and Feature Engineering:
    # Let's do some pre-processing and remove the last col, and the ids. We should also convert the diagnosis to ints
    data.drop(data.columns[[-1, 0]], axis=1, inplace=True)
    diagnosis = {'M':1.0, 'B':-1.0}
    data['diagnosis'] = data['diagnosis'].map(diagnosis)

    # Let's split the data up into features and labels, and normalize the features.
    Y = data.loc[:, 'diagnosis']  # labels
    X = data.iloc[:, 1:] # features
    
    # V2: filter features
    columns_dropped = remove_correlated_features(X)
    columns_dropped = remove_less_significant_features(X, Y)
    print(f'{len(columns_dropped)} columns dropped')
    
    # Normalize the features using MinMaxScalar from sklearn.preprocessing
    X_normalized = MinMaxScaler().fit_transform(X.values)
    X = pd.DataFrame(X_normalized)
    X.head()

    # Add the intercept col (b) and split the data (train/test/validate : 70/20/10)
    X.insert(loc=len(X.columns), column='intercept', value=1)
    X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)
    
    # Split once more for unseen data-subset
    #X_train, X_validate, y_train, y_validate = train_test_split(X_train, y_train, test_size=0.125, random_state=1)
    
    # Train the model
    print("training started...")
    W = sgd(X_train.to_numpy(), y_train.to_numpy())
    print("training finished.")
    
    # Test the model
    y_test_predicted = np.array([])
    for i in range(X_test.shape[0]):
        yp = np.sign(np.dot(W, X_test.to_numpy()[i])) #model
        y_test_predicted = np.append(y_test_predicted, yp)
        
    print("accuracy on test dataset: {}".format(accuracy_score(y_test.to_numpy(), y_test_predicted)))
    print("recall on test dataset: {}".format(recall_score(y_test.to_numpy(), y_test_predicted)))
    print("precision on test dataset: {}".format(recall_score(y_test.to_numpy(), y_test_predicted)))
           
# Send it 
reg_strength = 10000 
learning_rate = 0.000001
init()

# This returned
# accuracy on test dataset: 0.9736842105263158
# recall on test dataset: 0.9285714285714286
# precision on test dataset: 0.9285714285714286
# time 5000 epochs
# We will try to improve this by updating somethings above, which I will mark with comments "V2:",
# we will seek to remove some correlated (linear dependence) features depending on their "p-value" which 
# more-or-less rate the importance of a feature in the variation of the dependent variable "y". We will be removing 
# features with high p-values, keeping them makes the model more complex than need be. The new functions are:
# (1) remove_correlated_features()
# (2) remove_less_significant_features()

# Round 2 returned better results!
# accuracy on test dataset: 0.9824561403508771
# recall on test dataset: 0.9534883720930233
# precision on test dataset: 0.9534883720930233
# time 2048 epochs

8 columns dropped
training started...
Epoch is:1 and Cost is: 7226.382984689499
Epoch is:2 and Cost is: 6582.151387643106
Epoch is:4 and Cost is: 5492.0023941922145
Epoch is:8 and Cost is: 3859.169792319114
Epoch is:16 and Cost is: 2667.946213857656
Epoch is:32 and Cost is: 1976.6726887131326
Epoch is:64 and Cost is: 1593.5051552774753
Epoch is:128 and Cost is: 1326.9398558326545
Epoch is:256 and Cost is: 1160.05674083452
Epoch is:512 and Cost is: 1080.8734052929533
Epoch is:1024 and Cost is: 1048.8004903259514
Epoch is:2048 and Cost is: 1042.7716240571858
training finished.
accuracy on test dataset: 0.9824561403508771
recall on test dataset: 0.9534883720930233
precision on test dataset: 0.9534883720930233
